In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
from keras import layers
import numpy as np
import gym

In [2]:
from vae_recurrent import VAE, create_decoder, create_encoder
from transition_gru import TransitionGRU
from recurrent_agent import DAIFAgentRecurrent

In [3]:
from util import random_observation_sequence, transform_observations

In [4]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])
# tran = TransitionGRU(2, 1)

env = gym.make('MountainCarContinuous-v0')

Metal device set to: Apple M1 Pro


2022-08-19 11:26:28.491365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-19 11:26:28.491487: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
num_seqs = 1200
seq_length = 12
ob_seqs = []
next_obs = []

for i in range(num_seqs):
    o, a, r = random_observation_sequence(env, seq_length)

    train = np.concatenate([o[:-1], a], axis=1)
    test = o[-1]

    ob_seqs.append(train)
    next_obs.append(test)

ob_seqs = np.array(ob_seqs)
next_obs = np.array(next_obs)
ob_seqs.shape

ob_seqs_stddev = np.ones_like(ob_seqs)
next_obs_stddev = np.ones_like(next_obs)

next_obs

array([[-0.47365513, -0.00634077],
       [-0.5136172 , -0.0075119 ],
       [-0.5437829 ,  0.00712303],
       ...,
       [-0.505069  ,  0.00333155],
       [-0.58311284, -0.00410281],
       [-0.4861492 ,  0.00645138]], dtype=float32)

In [6]:
m = TransitionGRU(2, 1, 10, 30, 2)

m.compile(optimizer="Adam")
# m.build((None, None, 3))
# m.summary()
init_state = np.zeros((1200, 30))

In [7]:
m.fit((ob_seqs, init_state), (next_obs, next_obs_stddev), batch_size=24, epochs=20)

Epoch 1/20
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


2022-08-19 11:26:33.571106: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-19 11:26:33.981691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-19 11:26:34.103053: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 7/50 [===>..........................] - ETA: 0s - kl_loss: 0.1555 

2022-08-19 11:26:34.216726: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 1s 8ms/step - kl_loss: 0.0569
Epoch 2/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 9.3961e-04
Epoch 3/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 2.4292e-04
Epoch 4/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 1.9207e-04
Epoch 5/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 1.5777e-04
Epoch 6/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 1.2983e-04
Epoch 7/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 1.0718e-04
Epoch 8/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 9.1091e-05
Epoch 9/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 7.4817e-05
Epoch 10/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 6.3907e-05
Epoch 11/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 5.4703e-05
Epoch 12/20
50/50 [==============================] - 0s 7ms/step - 

In [32]:
res = m((ob_seqs[0:1], None))
res[3]

<tf.Tensor: shape=(1, 12, 30), dtype=float32, numpy=
array([[[-0.04653465, -0.04555459, -0.1264562 ,  0.09157347,
          0.08645334, -0.01161919, -0.03124051,  0.00628531,
         -0.01876677, -0.08953948,  0.01370354, -0.0010269 ,
          0.13280791, -0.0102782 , -0.05815428,  0.07444919,
          0.04029816, -0.07622987,  0.06161459, -0.00730191,
          0.06826576,  0.10570525,  0.03631176,  0.01300733,
         -0.07617977, -0.08929664,  0.07647785, -0.07727528,
         -0.01774083,  0.07626397],
        [-0.02689319, -0.11230602, -0.11106204,  0.05868537,
          0.10455961,  0.02713206,  0.13228741, -0.02897012,
         -0.05533361, -0.07269165,  0.02899782,  0.06728062,
          0.09858271, -0.04127914, -0.07770452,  0.12069084,
          0.02927887, -0.09142958,  0.06304795, -0.02826957,
          0.0659746 ,  0.103673  ,  0.11727359,  0.07382652,
         -0.01626572, -0.04237746,  0.0559689 , -0.06688105,
          0.05376701,  0.09337547],
        [-0.01033003,

In [35]:
tf.squeeze(res[3])

<tf.Tensor: shape=(12, 30), dtype=float32, numpy=
array([[-0.04653465, -0.04555459, -0.1264562 ,  0.09157347,  0.08645334,
        -0.01161919, -0.03124051,  0.00628531, -0.01876677, -0.08953948,
         0.01370354, -0.0010269 ,  0.13280791, -0.0102782 , -0.05815428,
         0.07444919,  0.04029816, -0.07622987,  0.06161459, -0.00730191,
         0.06826576,  0.10570525,  0.03631176,  0.01300733, -0.07617977,
        -0.08929664,  0.07647785, -0.07727528, -0.01774083,  0.07626397],
       [-0.02689319, -0.11230602, -0.11106204,  0.05868537,  0.10455961,
         0.02713206,  0.13228741, -0.02897012, -0.05533361, -0.07269165,
         0.02899782,  0.06728062,  0.09858271, -0.04127914, -0.07770452,
         0.12069084,  0.02927887, -0.09142958,  0.06304795, -0.02826957,
         0.0659746 ,  0.103673  ,  0.11727359,  0.07382652, -0.01626572,
        -0.04237746,  0.0559689 , -0.06688105,  0.05376701,  0.09337547],
       [-0.01033003, -0.17333214, -0.07171062,  0.01438317,  0.10286073,

In [9]:
next_obs[0:10]

array([[-0.47832373, -0.00274939],
       [-0.49251005, -0.005223  ],
       [-0.57791984, -0.00141938],
       [-0.47088268, -0.00629681],
       [-0.55019915, -0.00119392],
       [-0.45699415, -0.00656845],
       [-0.54774845,  0.00525958],
       [-0.4956426 ,  0.00690681],
       [-0.52634752,  0.00604456],
       [-0.47780034, -0.00620174]])

## Can we train just by single examples

In [69]:
num_seqs = 10
seq_length = 300
ob_seqs = []
next_obs = []

for i in range(num_seqs):
    o, a, r = random_observation_sequence(env, seq_length)

    train = np.concatenate([o[:-1], a], axis=1)
    test = o[1:]

    ob_seqs.append(train)
    next_obs.append(test)

ob_seqs = np.vstack(ob_seqs)
next_obs = np.vstack(next_obs)
ob_seqs.shape

ob_seqs_stddev = np.ones_like(ob_seqs)
next_obs_stddev = np.ones_like(next_obs)

ob_seqs_singles = ob_seqs.reshape(ob_seqs.shape[0], 1, ob_seqs.shape[1])
ob_seqs_seq = ob_seqs.reshape(1, ob_seqs.shape[0], ob_seqs.shape[1])
ob_seqs_seq.shape

(1, 3000, 3)

In [70]:
m = TransitionGRU(2, 1, 10, 30, 2)

m.compile(optimizer="Adam")
# m.build((None, None, 3))
# m.summary()
init_state = np.zeros((1200, 30))

In [73]:
epochs = 10

base_hidden_state = np.zeros((1, 30))

hidden_state = base_hidden_state

for i in range(epochs):

    _, _, _, h_states = m((ob_seqs_seq, base_hidden_state))

    # print(h_states)
    h_states = tf.squeeze(h_states)

    # exclude the last state as this will become the hidden state later on. next hidden state will become our new memory
    hidden_state = h_states[:-1]
    # next_hidden_state = h_states[-1]

    # add the current hidden state we saved to the start. This has h0, h1, h2, .. h=num_observations - 1
    hidden_state = tf.concat([base_hidden_state, hidden_state], axis=0)

    # print(hidden_state)
    # print(hidden_state.shape)

    m.fit((ob_seqs_singles, hidden_state), (next_obs, next_obs_stddev), batch_size=100, epochs=1)

30/30 [==============================] - 0s 6ms/step - kl_loss: 3.9038e-04


In [68]:
o, _ ,_,_ = m((ob_seqs_singles, hidden_state))
o[0:100]

<tf.Tensor: shape=(100, 2), dtype=float32, numpy=
array([[-2.0611016e-01, -4.8625730e-03],
       [-3.1964266e-01, -1.4081320e-03],
       [-3.9312947e-01,  4.0050014e-04],
       [-4.3926188e-01, -3.7089226e-04],
       [-4.6816337e-01, -2.3603828e-03],
       [-4.8863068e-01, -3.4953873e-03],
       [-4.9872929e-01, -3.6516972e-03],
       [-4.9581265e-01, -4.8003942e-04],
       [-5.0156003e-01, -4.1902158e-03],
       [-5.1178426e-01, -5.7753972e-03],
       [-5.1475662e-01, -4.9076127e-03],
       [-5.1557112e-01, -3.2200692e-03],
       [-5.2224922e-01, -5.0422531e-03],
       [-5.3257918e-01, -7.2859176e-03],
       [-5.4353613e-01, -9.1372002e-03],
       [-5.5619305e-01, -1.2263669e-02],
       [-5.6370258e-01, -5.9724916e-03],
       [-5.6467599e-01, -1.4274465e-03],
       [-5.6309086e-01,  6.2415225e-04],
       [-5.6267607e-01,  1.9366852e-03],
       [-5.6368721e-01,  3.0091251e-03],
       [-5.6426090e-01,  1.7140798e-03],
       [-5.6575900e-01,  3.1571402e-03],
       

In [65]:
next_obs

array([[-4.7439709e-01,  4.6088094e-06],
       [-4.7504199e-01, -6.4490089e-04],
       [-4.7657606e-01, -1.5340666e-03],
       ...,
       [-3.5934094e-01,  8.9467168e-03],
       [-3.5012558e-01,  9.2153521e-03],
       [-3.4338489e-01,  6.7406963e-03]], dtype=float32)

## Manually define a good policy

In [8]:
env = gym.make('MountainCarContinuous-v0')

env.reset()

while True:

    action = float(input())
    action = np.array([action])

    print(env.step(action))

(array([-0.46002594, -0.00198855], dtype=float32), -0.1, False, {})
(array([-0.4639884 , -0.00396246], dtype=float32), -0.1, False, {})
(array([-0.46989554, -0.00590716], dtype=float32), -0.1, False, {})
(array([-0.47770372, -0.00780819], dtype=float32), -0.1, False, {})
(array([-0.48735502, -0.00965132], dtype=float32), -0.1, False, {})
(array([-0.49877763, -0.01142261], dtype=float32), -0.1, False, {})
(array([-0.51188624, -0.0131086 ], dtype=float32), -0.1, False, {})
(array([-0.52658266, -0.01469642], dtype=float32), -0.1, False, {})
(array([-0.5427567 , -0.01617404], dtype=float32), -0.1, False, {})
(array([-0.5602871 , -0.01753044], dtype=float32), -0.1, False, {})
(array([-0.579043  , -0.01875583], dtype=float32), -0.1, False, {})
(array([-0.5988849 , -0.01984191], dtype=float32), -0.1, False, {})
(array([-0.61966693, -0.02078206], dtype=float32), -0.1, False, {})
(array([-0.6412384 , -0.02157148], dtype=float32), -0.1, False, {})
(array([-0.6634458 , -0.02220738], dtype=float32

ValueError: could not convert string to float: ''